In [ ]:
import os
import glob
import json
import shutil

# External Imports
import pandas as pd
import geopandas as gpd
import shapely

## Step 1: Convert the New Geojson File to CRS 4326
1. Enter the name of the geojson file into the variable **geojson_file**

In [ ]:
# enter the name of the geojson file
geojson_file = ""
# Example: geojson_file = "usa_southeast_transects_DE.geojson"

2. Move the geojson file into the main coastseg directory
- Manually drop the file into the coastseg directory
- After you moved the file there run the following block of code

In [ ]:
filepath = os.path.abspath(os.getcwd())+os.sep+geojson_file 
filepath

3. Convert geojson file to crs 4326
- Run the following code block to convert to epsg 4326

In [ ]:
# read the geojson file into a geodataframe
gdf = gpd.read_file(filepath)
# convert to new crs
gdf = gdf.to_crs("epsg:4326")
# overwrites the original file
gdf.to_file(filepath,driver="GeoJSON")

### Look at the geodataframe for the transects to see if it looks alright

In [ ]:
gdf

## Remove any z-axis geometeries

In [ ]:
def remove_z_axis(geodf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """If the geodataframe has z coordinates in any rows, the z coordinates are dropped.
    Otherwise the original geodataframe is returned.

    Additionally any multi part geometeries will be exploded into single geometeries.
    eg. MutliLineStrings will be converted into LineStrings.
    Args:
        geodf (gpd.GeoDataFrame): geodataframe to check for z-axis

    Returns:
        gpd.GeoDataFrame: original dataframe if there is no z axis. If a z axis is found
        a new geodataframe is returned with z axis dropped.
    """
    if geodf.empty:
        print(f"Empty geodataframe has no z-axis")
        return geodf

    # if any row has a z coordinate then remove the z_coordinate
    if geodf["geometry"].has_z.any():

        def remove_z_from_row(row):
            if row.geometry.has_z:
                row.geometry = shapely.ops.transform(
                    lambda x, y, z=None: (x, y), row.geometry
                )
                return row
            else:
                return row

        # Use explode to break multilinestrings in linestrings
        feature_exploded = geodf.explode(ignore_index=True)
        # For each linestring portion of feature convert to lat,lon tuples
        no_z_gdf = feature_exploded.apply(remove_z_from_row, axis=1)
        return no_z_gdf
    else:
        return geodf


In [ ]:
gdf = remove_z_axis(gdf)

In [ ]:
gdf

## Step 2. Create an ID column and drop unneeded columns
Transects in coastseg are identified by a unique id in the column 'id'


In [ ]:
def drop_cols_except_id_slope_geom(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.copy()

    # Ensure column names are all lowercase
    gdf.columns = [col.lower() for col in gdf.columns]

    # Identify columns to drop
    cols_to_drop = set(gdf.columns) - set(['geometry'])
    if 'id' in gdf.columns:
        cols_to_drop -= set(['id'])
    if 'slope' in gdf.columns:
        cols_to_drop -= set(['slope'])

    # Drop unneeded columns
    gdf.drop(columns=cols_to_drop, inplace=True)

    # Ensure 'id' and 'slope' columns exist in the GeoDataFrame
    if 'id' not in gdf.columns:
        gdf['id'] = None
    if 'slope' not in gdf.columns:
        gdf['slope'] = None

    # Rearrange columns
    cols = ['id', 'slope', 'geometry']
    gdf = gdf[[col for col in cols if col in gdf.columns]]

    # Drop the slope column if it's empty
    if gdf['slope'].isna().all():
        gdf.drop('slope', axis=1, inplace=True)

    return gdf


## Create unique ids for each transects
Only run this code if the id for each transect is not unique

In [ ]:
gdf['id'] = [i for i, _ in enumerate(gdf.index)]

## Drop any columns that won't be used
Always run this code

In [ ]:
if 'id' not in gdf.columns:
    gdf['id'] = [i for i, _ in enumerate(gdf.index)]
    
gdf = drop_cols_except_id_slope_geom(gdf)
gdf

## Step 3: Create a new Bounding Box for all the transects
1. Move the new geojson file into the transects directory
- Run the following code block

In [ ]:
# save the modified geodataframe to file
gdf.to_file(filepath,driver="GeoJSON")
# full path to transects directory which contains all the transect geojson files
transects_folder=os.path.join(os.getcwd(),"src","coastseg","transects")
destination = os.path.join(transects_folder,geojson_file)
# move the new geojson file to the transects directory
shutil.move(filepath, destination)

2. Make a list of the full paths to all the transect geojson files in the transects directory

In [ ]:
transects=glob.glob(transects_folder+os.sep+"*.geojson")
transects

3. Make a list of all the transect geojson filenames in the transects directory

In [ ]:
transect_layer_names=[os.path.basename(transect) for transect in transects]
transect_layer_names

4. Create a list of the total bounds for each transect geojson file
- This might take awhile because it has to open and close each geojson file

In [ ]:
transects_total_bounds=[gpd.read_file(transect_file).total_bounds for transect_file in transects]
transects_total_bounds

5. Create a dataframe with the total bounds and filename of each transects file

In [ ]:
df = pd.DataFrame(transects_total_bounds,columns=['minx', 'miny', 'maxx', 'maxy'])
df['filename'] = transect_layer_names
df

## Step 3: Save the dataframe to a csv file 
- Save the CSV file to the bounding_boxes directory

In [ ]:
csv_file = "transects_bounding_boxes.csv"
bounding_box_path = os.path.join(os.getcwd(),"src","coastseg","bounding_boxes")
csv_path = transect_folder=os.path.join(bounding_box_path,csv_file)
df.to_csv(csv_path,index=False)

## Step 4 (Optional): Open the csv file to verify it looks correct

In [ ]:
transects_df=pd.read_csv(csv_path)
transects_df